In [82]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install pyspark
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

In [83]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-01-26 19:58:12--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.3’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.37MB/s    in 0.2s    

2022-01-26 19:58:12 (5.37 MB/s) - ‘postgresql-42.2.9.jar.3’ saved [914037/914037]



In [84]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETLAmazon").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [85]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
watch_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True, timestampFormat="yyyy-mm-dd")
# Show DataFrame
watch_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...|2015-01-31 00:08:00|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...| 

In [86]:
#Count the number of records (rows) in the dataset.
print(watch_data_df.count())
watch_data_df=watch_data_df.dropDuplicates()
watch_data_df=watch_data_df.dropna()
print(watch_data_df.count())

960872
960679


**Transform the dataset to fit the tables in the schema file.**

In [87]:
# Load in a sql function to use columns
from pyspark.sql.functions import col,to_date

**Review ID Table**

In [88]:
review_id_table_df=watch_data_df.select(['review_id','customer_id','product_id','product_parent','review_date'])
print(review_id_table_df.count())
print(review_id_table_df.distinct().count())
review_id_table_df.show()

960679
960679
+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R1004K426LTDKN|   15688375|B002XUL6YY|     852892099|2014-01-22 00:05:00|
|R1006QGKXQ3Q96|   21739087|B005JVP0LE|     559701518|2014-01-16 00:06:00|
|R1017P0ZGRXSR0|    2501262|B0009P66YK|       6220965|2015-01-26 00:02:00|
|R103HXXDQD9W4Y|   16842563|B008MVVA2Q|     849986272|2014-01-30 00:12:00|
|R10472PWW2S2AW|   52130705|B001D9FGAG|     940154187|2009-01-27 00:01:00|
|R104EYLG2ZXXJ1|   41518340|B007I1PLUI|     906701484|2014-01-27 00:11:00|
|R105KF16VGOILR|   48845798|B0041IEATO|     950717807|2011-01-02 00:09:00|
|R106A2ETV5NZNE|   50381801|B000JQSMQO|     963972840|2012-01-16 00:12:00|
|R106UPPPFIIX3P|   42038724|B001CTI5SM|     243892088|2013-01-17 00:08:00|
|R107IKXXNUAZZC|    1594613|B00JMPN3RQ|     849006421|2015-01-11 00:08:00|
|R1081CHIZX

**Customer Table**


In [89]:
customer_df=watch_data_df.select(['customer_id'])
customer_df=customer_df.groupBy('customer_id').count()
customer_df = customer_df.withColumnRenamed("count", "customer_count") 
customer_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   45353385|             1|
|    7166154|             1|
|   27053878|             1|
|   33656295|             1|
|   34320055|             1|
|   12611685|             1|
|   26417168|             5|
|   16885399|             3|
|   49090853|             1|
|   48080283|             1|
|   12937506|             5|
|   42209526|             1|
|   36125184|             1|
|   50308356|             1|
|   13273143|             1|
|   30675744|             1|
|   10196801|             3|
|   10748967|             1|
|   44443429|             1|
|   15687327|             2|
+-----------+--------------+
only showing top 20 rows



**Product Table**

In [90]:
products_df=watch_data_df.select(['product_id','product_title'])
print(products_df.count())
print(products_df.distinct().count())
products_df=products_df.dropDuplicates()
print(products_df.count())
products_df.show()

960679
149694
149694
+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00EVX7V1I|Game Time Women's...|
|B009S4DODY|XOXO Women's XO55...|
|B00LBKXQRW|Anne Klein Women'...|
|B0009P679Y|Invicta Men's 993...|
|B00DHF30RU|M&c Women's | Cla...|
|B00NIDA43Y|GuTe Classic Skel...|
|B008EQDDPQ|Nautica Men's N13...|
|B004VRBZ66|Timex Men's T2N63...|
|B009BEO81I|        Fossil Riley|
|B008B39MTI|XOXO Women's XO55...|
|B00TGPM8PU|Handmade Wooden W...|
|B00VNXQQQ0|Eterna 2520-41-64...|
|B00B1PV1C4|Nautica Men's N19...|
|B00N1Y8TQ4|Tissot Men's T095...|
|B00G6DBTY6|red line Men's RL...|
|B00HM04AYI|Columbia Men's Fi...|
|B00VI8HB96|GUESS I90176L1 Wo...|
|B00IT25WJU|LanTac DGN556B Dr...|
|B0106S12XE|Skmei S Shock Ana...|
|B00FPSJ63Y|Michael Kors Ladi...|
+----------+--------------------+
only showing top 20 rows



**Vine Table**

In [91]:
vine_table_df=watch_data_df.select(['review_id','star_rating','helpful_votes','total_votes','vine'])
print(vine_table_df.count())
print(vine_table_df.distinct().count())
vine_table_df.show()

960679
960679
+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1004K426LTDKN|          2|            0|          1|   N|
|R1006QGKXQ3Q96|          5|            0|          0|   N|
|R1017P0ZGRXSR0|          5|            0|          0|   N|
|R103HXXDQD9W4Y|          5|            0|          0|   N|
|R10472PWW2S2AW|          4|            0|          0|   N|
|R104EYLG2ZXXJ1|          1|            0|          0|   N|
|R105KF16VGOILR|          4|            1|          1|   N|
|R106A2ETV5NZNE|          5|            0|          0|   N|
|R106UPPPFIIX3P|          5|            0|          0|   N|
|R107IKXXNUAZZC|          5|            0|          0|   N|
|R1081CHIZX3PZW|          4|            1|          1|   N|
|R1092B4NE0WNDD|          4|            0|          0|   N|
|R109WVVOK26661|          1|            0|          0|   N|
|R10AQ92VE5HTJ6|          

**Loading to Database**

In [92]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<connectionstring>:5432/Amazon_Review"
config = {"user":"postgres", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [94]:
review_id_table_df.write.jdbc(url=jdbc_url,table='review_id_table',mode=mode,properties=config)

In [95]:
customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [96]:
products_df.write.jdbc(url=jdbc_url,table='products',mode=mode,properties=config)

In [97]:
vine_table_df.write.jdbc(url=jdbc_url,table='vine_table',mode=mode,properties=config)

# New Section